In [26]:
from __future__ import print_function
from urllib2 import HTTPError
from urllib import quote
from urllib import urlencode
from pprint import pprint

import argparse
import json
import pprint
import requests
import sys
import urllib
import time

import pandas as pd
import numpy as np

In [27]:
# OAuth credential placeholders
CLIENT_ID = 'sd5Ge9lqAHFdxK_wSPp3QA'
CLIENT_SECRET = 'DRLcojBT3IAM1kVJDViB6QOHmjEIHEubFTpqHdfLHoEiTJSgEFt5Peqzbcajve7K'


In [28]:
# API constants, you shouldn't have to change these.
API_HOST = 'https://api.yelp.com'
SEARCH_PATH = '/v3/businesses/search'
BUSINESS_PATH = '/v3/businesses/'  # Business ID will come after slash.
TOKEN_PATH = '/oauth2/token'
GRANT_TYPE = 'client_credentials'

In [29]:
url = 'https://api.yelp.com/oauth2/token'
payload = {'grant_type':'client_credentials','client_id':'sd5Ge9lqAHFdxK_wSPp3QA','client_secret':'DRLcojBT3IAM1kVJDViB6QOHmjEIHEubFTpqHdfLHoEiTJSgEFt5Peqzbcajve7K'}

r = requests.post(url, data=payload)

In [30]:
r.text

u'{"expires_in": 15325906, "token_type": "Bearer", "access_token": "n2C-ZzYcr5oDGNJBiHiqLGfFBtHi6cKcgDDuQNyrHXU8pgwmyaH2hZ54C4OD4L7OyzONVS2UTbPqjsDsrW_QXNIOkOfoSjTS5dvW9OeMUXFoOj6qdW5jSGq4nsdMWHYx"}'

In [31]:
TOKEN = "n2C-ZzYcr5oDGNJBiHiqLGfFBtHi6cKcgDDuQNyrHXU8pgwmyaH2hZ54C4OD4L7OyzONVS2UTbPqjsDsrW_QXNIOkOfoSjTS5dvW9OeMUXFoOj6qdW5jSGq4nsdMWHYx"
#return 5 results by default
SEARCH_LIMIT = 1

In [32]:
def request(host, path, bearer_token, url_params=None):
    """Given a bearer token, send a GET request to the API.
    Args:
        host (str): The domain host of the API.
        path (str): The path of the API after the domain.
        bearer_token (str): OAuth bearer token, obtained using client_id and client_secret.
        url_params (dict): An optional set of query parameters in the request.
    Returns:
        dict: The JSON response from the request.
    Raises:
        HTTPError: An error occurs from the HTTP request.
    """
    url_params = url_params or {}
    url = '{0}{1}'.format(host, quote(path.encode('utf8')))
    headers = {
        'Authorization': 'Bearer %s' % bearer_token,
    }

    print(u'Querying {0} ...'.format(url))

    response = requests.request('GET', url, headers=headers, params=url_params)

    return response.json()

In [33]:
def search(bearer_token, term, location):
    """Query the Search API by a search term and location.
    Args:
        term (str): The search term passed to the API.
        location (str): The search location passed to the API.
    Returns:
        dict: The JSON response from the request.
    """

    url_params = {
        'term': term.replace(' ', '+'),
        'location': location.replace(' ', '+'),
        'limit': SEARCH_LIMIT
    }
    return request(API_HOST, SEARCH_PATH, bearer_token, url_params=url_params)

In [34]:
#test the search function
response = search(TOKEN, 'dominos', 'chicago')

Querying https://api.yelp.com/v3/businesses/search ...


In [35]:
response

{u'businesses': [{u'categories': [{u'alias': u'pizza', u'title': u'Pizza'},
    {u'alias': u'chicken_wings', u'title': u'Chicken Wings'},
    {u'alias': u'sandwiches', u'title': u'Sandwiches'}],
   u'coordinates': {u'latitude': 41.9244149, u'longitude': -87.7420833},
   u'distance': 5740.245410879999,
   u'id': u'dominos-pizza-chicago-11',
   u'image_url': u'',
   u'is_closed': False,
   u'location': {u'address1': u'4608 W Fullerton Ave',
    u'address2': u'',
    u'address3': u'',
    u'city': u'Chicago',
    u'country': u'US',
    u'state': u'IL',
    u'zip_code': u'60639'},
   u'name': u"Domino's Pizza",
   u'phone': u'+17737729000',
   u'price': u'$$',
   u'rating': 3.5,
   u'review_count': 19,
   u'url': u'https://www.yelp.com/biz/dominos-pizza-chicago-11?adjust_creative=sd5Ge9lqAHFdxK_wSPp3QA&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=sd5Ge9lqAHFdxK_wSPp3QA'}],
 u'total': 104}

In [36]:
#test how to get info out of json file
price = []
rating = []
estab_price = response['businesses'][0]["price"]
estab_rating = response['businesses'][0]["rating"] 
price.append(estab_price)
rating.append(estab_rating)
price

[u'$$']

In [38]:
#let's mess with our data...
insp = pd.read_csv('C:/Users/Joeseph/Dropbox/HGSE/CS109a Data Science/final project/data/Food_Inspections_Clean2.csv')
insp['date'] = pd.to_datetime(insp['Inspection Date'].astype('str'),infer_datetime_format=True)
insp.loc[:,'day_of_week']=insp.date.dt.dayofweek.values
insp.loc[:,'day_of_year']=insp.date.dt.dayofyear.values
insp.rename(columns={'AKA Name':'name','License #':'license','Inspection ID':'ID','Facility Type':'facility','Inspection Type':'inspection_type'},
            inplace=True)
insp = insp.query('facility in ["Restaurant","Grocery Store","Bakery"]')
insp = insp[['name','ID','license','date','Results','Risk','facility','inspection_type',
             'month','day','year','day_of_week','day_of_year','Latitude','Longitude','Zip']]
insp.head()

,name,ID,license,date,Results,Risk,facility,inspection_type,month,day,year,day_of_week,day_of_year,Latitude,Longitude,Zip
0,IYANZE,1970470,1909713.0,2016-10-28,1.0,1.0,Restaurant,Short Form Complaint,10,28,2016,4,302,41.966063,-87.657734,60640.0
2,DOMINOS,1970461,2464564.0,2016-10-28,0.0,2.0,Restaurant,Complaint Re-Inspection,10,28,2016,4,302,41.866277,-87.639360,60607.0
4,FIRST SLICE,1970429,1620340.0,2016-10-28,2.0,1.0,Restaurant,Complaint,10,28,2016,4,302,41.961619,-87.673833,60640.0
5,MCDONALDS,1951297,2349672.0,2016-10-28,0.0,2.0,Restaurant,Complaint,10,28,2016,4,302,41.883717,-87.626086,60602.0
6,PIZZA Y PAN PA' YA CORP.,1970418,2496099.0,2016-10-28,0.0,1.0,Restaurant,License,10,28,2016,4,302,41.961086,-87.706726,60618.0


In [39]:
#make sure all the names are strings (got a float error once...)
insp['name'] = insp['name'].astype(str)

In [40]:
n = insp.shape[0]
price = np.zeros(n)
rating = np.zeros(n)
insp['price'] = price
insp['rating'] = rating

In [44]:
%%time

for i in range(16310,n):
    print('Results for row {}:'.format(i))
    response = search(TOKEN, insp.iloc[i,0], 'chicago')
    time.sleep(1)
    if response['total'] == 0:
        insp.iloc[i,16] = 0
        insp.iloc[i,17] = 0
    elif 'price' not in (response['businesses'][0] or response['businesses']):
        insp.iloc[i,16] = 0
        insp.iloc[i,17] = 0
    else:
        insp.iloc[i,16] = response['businesses'][0]["price"]
        insp.iloc[i,17] = response['businesses'][0]["rating"]

Results for row 16310:
Querying https://api.yelp.com/v3/businesses/search ...
Results for row 16311:
Querying https://api.yelp.com/v3/businesses/search ...
Results for row 16312:
Querying https://api.yelp.com/v3/businesses/search ...
Results for row 16313:
Querying https://api.yelp.com/v3/businesses/search ...
Results for row 16314:
Querying https://api.yelp.com/v3/businesses/search ...
Results for row 16315:
Querying https://api.yelp.com/v3/businesses/search ...
Results for row 16316:
Querying https://api.yelp.com/v3/businesses/search ...
Results for row 16317:
Querying https://api.yelp.com/v3/businesses/search ...
Results for row 16318:
Querying https://api.yelp.com/v3/businesses/search ...


ValueError: No JSON object could be decoded

In [45]:
insp.to_csv('insp_yelp.csv', sep=',')

In [50]:
insp_yelp = pd.read_csv('insp_yelp.csv', sep = ',')
insp_yelp.head()

C:\Users\Joeseph\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,name,ID,license,date,Results,Risk,facility,inspection_type,month,day,year,day_of_week,day_of_year,Latitude,Longitude,Zip,price,rating
0,IYANZE,1970470,1909713.0,10/28/2016,1,1.0,Restaurant,Short Form Complaint,10,28,2016,4,302,41.966063,-87.657734,60640.0,$$,3.0
1,DOMINOS,1970461,2464564.0,10/28/2016,0,2.0,Restaurant,Complaint Re-Inspection,10,28,2016,4,302,41.866277,-87.639360,60607.0,$$,3.5
2,FIRST SLICE,1970429,1620340.0,10/28/2016,2,1.0,Restaurant,Complaint,10,28,2016,4,302,41.961619,-87.673833,60640.0,$,4.5
3,MCDONALDS,1951297,2349672.0,10/28/2016,0,2.0,Restaurant,Complaint,10,28,2016,4,302,41.883717,-87.626086,60602.0,$,3.0
4,PIZZA Y PAN PA' YA CORP.,1970418,2496099.0,10/28/2016,0,1.0,Restaurant,License,10,28,2016,4,302,41.961086,-87.706726,60618.0,$$$,4.5


In [52]:
insp_yelp['price'] = insp_yelp['price'].astype(str)
dollars = {'0': '0',
           '0.0': '0',
           '1': '1',
           '2': '2',
           '3': '3',
           '4': '4',
           '5': '5',
           '$': '1',
           '$$': '2',
           '$$$': '3',
           '$$$$': '4',
           '$$$$$': '5'}
for i in range(insp_yelp.shape[0]):
    insp_yelp.iloc[i,16] = dollars[insp_yelp.iloc[i,16]]

In [48]:
insp_yelp.price.value_counts().head(10)

0    96184
2      293
1      229
3       31
4        7
Name: price, dtype: int64

In [21]:
insp_yelp = insp_yelp[insp_yelp.price != '0']
insp_yelp.price.value_counts().head(10)

1    5874
2    5724
3     766
4     272
Name: price, dtype: int64

In [22]:
insp_yelp['price'] = insp_yelp['price'].astype(int)

In [53]:
insp_yelp.rating.value_counts().head(10)

0.0    84109
4.0     4252
3.5     3149
4.5     1884
3.0     1812
5.0      742
2.5      562
2.0      153
1.5       49
1.0       32
Name: rating, dtype: int64

In [25]:
insp_yelp.to_csv('insp_yelp_clean.csv', sep=',')